# Категориальные признаки

Снова будем использовать немного видоизменённый для наших целей датасет https://archive.ics.uci.edu/ml/datasets/AutoUniv.

В нём присутствуют целочисленные, вещественнозначные и категориальные признаки.

In [ ]:
import numpy as np
import pandas as pd

df = pd.read_csv('table.csv')
df.head()

Найдем все категориальные признаки

In [ ]:
cat_features_mask = (df.dtypes == "object").values
cat_features_mask

In [ ]:
len(cat_features_mask[cat_features_mask==True])

**Перевод текстовых категориальных признаков в числа**

Переведем все нечисловые признаки в числовые (порядковые), а затем будем их кодировать различными способами.

In [ ]:
from sklearn import preprocessing

label_enc = preprocessing.LabelEncoder()
for feature in df.columns[cat_features_mask]: 
    df[feature] = label_enc.fit_transform(df[feature])
df.head()

**OneHot-кодирование**

Пусть некоторый признак принимает значения из множества K. OneHotEncoder вместо одного признака создает K бинарных признаков - по одному на каждое возможное значение исходного признака.

In [ ]:
enc = preprocessing.OneHotEncoder(sparse=False)
df_cat = enc.fit_transform(df[df.columns[cat_features_mask]])
df_cat = pd.DataFrame(data=df_cat)

In [ ]:
print df_cat.shape
df_cat.head()

**Хэширование**

HashingVectorizer преобразовывает строку в числовой массив заданной длиной с помощью хэш-функции. В этом методе в качестве входных параметров мы задаем желаемое количество новых признаков, а также токенизатор - обработчик текста (в нём мы можем сделать любую удобную нам предобработку текста: удалить редкие слова, удалить знаки препинания, оставить только слова из определенного списка и т.д.). Токенизатор возвращает текст, разбитый на токены, т.е. на слова.

In [ ]:
def my_tokenizer(s):
    return [elem for elem in s.split()]

Наиболее интересный для нас с точки зрения хэширования - столбец att9.

In [ ]:
df = pd.read_csv('table.csv')

#Выведите на экран все различные значения элементов из столбца 'att9'
#Your code is here

Для применения HashingVectorize выбираем из столбца все различные значения(слова) без повторений, обучаем HashingVectorizer на этих словах и применяем ко всему столбцу. В итоге мы получаем разреженную матрицу. С ней умеют работать многие алгоритмы машинного обучения, но при желании можем перевести ее в numpy array.

In [ ]:
from sklearn.feature_extraction.text import HashingVectorizer

coder = HashingVectorizer(tokenizer=my_tokenizer, n_features=15)

TrainNotDuble = df['att9'].drop_duplicates()
coder.fit(TrainNotDuble)

coder.transform(df['att9'].values).toarray()

# Работа с текстами

In [ ]:
from tqdm import tqdm
import regex as re

%pylab inline

Одним из направлений машинного обучения является работа с текстами и извлечение полезной информации из текстов. Чтобы алгоритмы машинного обучения могли работать с текстами, необходимо перевести тексты в наборы чисел. Для этого применяют различные алгоритмы векторизации текстов. 

Будем изучать датасет, содержащий отзывы о фильмах. Отзывы могут быть положительные, либо отрицательные. Наша конечная задача - научиться различать положительные и отрицательные отзывы.

Загрузим датасет и уберем из него плохие строки (в которых нет оценки фильму).

In [ ]:
responses = []
X = []
y = []
import codecs 

bad = 0
with codecs.open('imdb_labelled.txt',encoding='utf-8') as thefile:
    for row in tqdm(thefile.readlines()):
        try:
            resp_curr, y_curr = row.split('\t')
        except:
            'ValueError'
            print row
            bad+=1
        X.append(resp_curr)
        y.append(int(y_curr))
bad

In [ ]:
bad_responses = filter(lambda r: 'awful' in r, X)
print bad_responses[1]

#Выведите на экран несколько строк со словами 'bad' и 'great'. 
#Верно ли, что все отзывы со словом 'bad' негативные, а отзывы со словом 'great' позитивные?
#Your code is here

Первые этапы обработки текста:

* снижение регистра

* удаление пунктуации

* удаление всех символов, кроме символов нашего алфавита (в данном случае, латинского)

In [ ]:
print re.sub(ur'[^\p{Latin}]', ' ', bad_responses[1].lower())

In [ ]:
Texts = map(lambda r: re.sub(ur'[^\p{Latin}]', ' ', r.lower()), X)
Texts[35]

Посмотрим на распределение ответов в наших данных. 

In [ ]:
hist(y)

**1 способ векторизации: счётчик (CountVectorizer)**

Каждому слову соответствует количество его вхождений в текст.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
vectorizer = CountVectorizer(encoding='utf8', min_df=5)
_ = vectorizer.fit(Texts)
vectorizer.transform(Texts[:1])

In [ ]:
print vectorizer.transform(Texts[:1]).indptr
print vectorizer.transform(Texts[:1]).indices
print vectorizer.transform(Texts[:1]).data

In [ ]:
from IPython.display import display, Math, Latex

**2 способ векторизации: TF-IDF**

Ещё один способ работы с текстовыми данными — TF-IDF (Term Frequency–Inverse Document Frequency). Рассмотрим коллекцию текстов $D$. Для каждого уникального слова $t$ из документа $d \in D$ вычислим следующие величины:

1. Term Frequency – количество вхождений слова в отношении к общему числу слов в тексте: 
    $$\text{tf}(t, d) = \frac{n_{td}}{\sum_{t \in d} n_{td}},$$ где $n_{td}$ — количество вхождений слова $t$ в текст $d$.
2. Inverse Document Frequency $$\text{idf}(t, D) = \log \frac{\left| D \right|}{\left| \{d\in D: t \in d\} \right|},$$ где $\left| \{d\in D: t \in d\} \right|$ – количество текстов в коллекции, содержащих слово $t$.

Тогда для каждой пары (слово, текст) $(t, d)$ вычислим величину: $$\text{tf-idf}(t,d, D) = \text{tf}(t, d)\cdot \text{idf}(t, D).$$

Отметим, что значение $\text{tf}(t, d)$ корректируется для часто встречающихся общеупотребимых слов при помощи значения $\text{idf}(t, D).$

Признаковым описанием одного объекта $d \in D$ будет вектор $\bigg(\text{tf-idf}(t,d, D)\bigg)_{t\in V}$, где $V$ – словарь всех слов, встречающихся в коллекции $D$.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
#Объявите TrIdfVectorizer с теми же параметрами, что и CountVectorizer
#Your code is here
vectorizer = TfidfVectorizer(...)

#Сделайте fit и predict, как было сделано в предыдущем примере
#Your code is here

In [ ]:
print vectorizer.transform(Texts[:1]).indptr
print vectorizer.transform(Texts[:1]).indices
print vectorizer.transform(Texts[:1]).data

Применим два рассмотренных метода векторизации к задаче классификации отзывов на два класса (положительные и отрицательные).

In [ ]:
from sklearn.cross_validation import ShuffleSplit
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, accuracy_score

In [ ]:
vectorizer = CountVectorizer(encoding='utf8', min_df=5)
_ = vectorizer.fit(Texts)

X = vectorizer.transform(Texts)
y = np.array(y)

cv = ShuffleSplit(X.shape[0], n_iter=1, test_size=0.2)

for train_ids, test_ids in cv:
    lr = LogisticRegression()
    lr.fit(X[train_ids], y[train_ids])
    preds = lr.predict_proba(X[test_ids])[:,1]
    print 'ROC-AUC: %.3f, ACC: %.3f' % (roc_auc_score(y[test_ids], preds), 
                                        accuracy_score(y[test_ids], (preds > 0.5).astype(int)))

In [ ]:
vectorizer = TfidfVectorizer(encoding='utf8', min_df=5)
_ = vectorizer.fit(Texts)

X = vectorizer.transform(Texts)
y = np.array(y)

for train_ids, test_ids in cv:
    lr = LogisticRegression()
    lr.fit(X[train_ids], y[train_ids])
    preds = lr.predict_proba(X[test_ids])[:,1]
    print 'ROC-AUC: %.3f, ACC: %.3f' % (roc_auc_score(y[test_ids], preds), 
                                        accuracy_score(y[test_ids], (preds > 0.5).astype(int)))

**Важность признаков**

В задачах, связанных с обработкой текстов, признаки как правило хорошо интерпретируемы. Для визуального контроля качества работы алгоритма можно посмотреть на те слова, которые алгоритм посчитал наиболее важными для данной задачи.

In [ ]:
weights = zip(vectorizer.get_feature_names(), lr.coef_[0])
weights = sorted(weights, key=lambda i: i[1])
for i in range(1,20):
    print '%s, %.2f' % weights[-i]
    
print '...'
for i in reversed(range(1,20)):
    print '%s, %.2f' % weights[i]

**3 способ векторизации: Word2Vec**

Word2Vec - это алгоритм, который собирает статистику по совместному появлению слов в фразах, а затем с помощью нейронных сетей решает задачу снижения размерности и выдает на выходе компактные векторные представления слов, в максимальной степени отражающие отношения этих слов в обрабатываемых текстах.

Нахождение связей между контекстами слов основано на предположении, что слова, находящиеся в похожих контекстах, имеют тенденцию значить похожие вещи, т.е. быть семантически близкими. 
Формально задача стоит так: максимизировать косинусное расстояние между векторами слов (скалярное произведение векторов), которые появляются рядом друг с другом, и минимизировать косинусное расстояние между векторами слов, которые не появляются друг рядом с другом. Рядом друг с другом в данном случае значит в близких контекстах.

Применим векторизацию с помощью word2vec для наших данных. Кроме того, удалим stop-слова, то есть слова, часто встречающиеся во всех английских текстах - это ещё один полезный метод обработки текстов.

In [ ]:
from nltk.corpus import stopwords

stops = set(stopwords.words("english"))
stops

In [ ]:
def delete_stopwords(review, remove_stopwords=True):
    
    words = review.split()

    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    return(words)

Переведем строки нашего датасета в токенизированный вид и удалим из них стоп-слова - в этом виде они пригодны для использования word2vec.

In [ ]:
import nltk.data
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

def review_to_sentences(review,tokenizer,remove_stopwords=True ):

    raw_sentences = tokenizer.tokenize(review.strip())

    sentences = []
    for sentence in raw_sentences:
        if len(sentence) > 0:
            sentences.append(delete_stopwords(sentence,remove_stopwords))
    return sentences

sentences = []
Y = []
for i in range(len(Texts)):
    if len(set(Texts[i])) == 1:
        continue
    Y.append(y[i])
    sentences += review_to_sentences(Texts[i], tokenizer)

Применим word2vec к токенизированному корпусу.

In [ ]:
num_features = 500                       
min_word_count = 5                   
num_workers = 4       
context = 10                                                                               
downsampling = 1e-5   

from gensim.models import word2vec

model = word2vec.Word2Vec(sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

model.init_sims(replace=True)

model_name = str(num_features)+"features_word2vec"
model.save(model_name)

Теперь каждое слово корпуса имеет векторное представление

In [ ]:
model.wv['good']

В качестве одного из способов векторизовать текст (в нашем случае отзыв на фильм), можно усреднить векторы слов, входящих в этот текст. Так и сделаем.

In [ ]:
import numpy as np
import re
from nltk.corpus import stopwords

index2word_set = set(model.wv.index2word)

def make_featurevec(words, model, num_features):
    featureVec = np.zeros((num_features,),dtype="float32")
    nwords = 0.
    for word in words:
        if word in index2word_set:
            nwords = nwords + 1.
            featureVec += model[word]
    if nwords > 0:
        featureVec = np.divide(featureVec,nwords)
    return featureVec

def get_avg_featurevecs(reviews, model, num_features):
    counter = 0
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    for review in reviews:
        reviewFeatureVecs[counter] = make_featurevec(review, model, num_features)
        counter = counter + 1
    return reviewFeatureVecs

trainDataVecs = get_avg_featurevecs(sentences, model, num_features)

Наконец, обучим классификатор на полученных признаках.

In [ ]:
cv = ShuffleSplit(trainDataVecs.shape[0], n_iter=1, test_size=0.2)

Y = np.array(Y)

for train_ids, test_ids in cv:
    lr = LogisticRegression()
    lr.fit(trainDataVecs[train_ids], Y[train_ids])
    preds = lr.predict_proba(trainDataVecs[test_ids])[:,1]
    print 'ROC-AUC: %.3f, ACC: %.3f' % (roc_auc_score(Y[test_ids], preds), 
                                        accuracy_score(Y[test_ids], (preds > 0.5).astype(int)))

In [ ]:
#Попробуйте обучить word2vec не на 500 признаках, а на 100, 250, 1000. Какое количество признаков дает наилучшее качество модели?
#Напишите здесь вывод.

# Визуализация кластеров слов с признаками из Word2Vec, с помощью t-SNE

In [ ]:
from gensim.models.word2vec import Word2Vec
from sklearn.manifold import TSNE
from sklearn.datasets import fetch_20newsgroups
import re
import matplotlib.pyplot as plt

train = fetch_20newsgroups()

def clean(text):
    """Remove posting header, split by sentences and words, keep only letters"""
    lines = re.split('[?!.:]\s', re.sub('^.*Lines: \d+', '', re.sub('\n', ' ', text)))
    return [re.sub('[^a-zA-Z]', ' ', line).lower().split() for line in lines]

sentences = [line for text in train.data for line in clean(text)]

model = Word2Vec(sentences, workers=4, size=100, min_count=50, window=10, sample=1e-3)

print (model.most_similar('day'))

In [ ]:
from nltk import word_tokenize
from nltk.corpus import stopwords
stop = set(stopwords.words('english'))

vocab = [elem for elem in model.wv.index2word if elem not in stop]
vocab = [elem for elem in vocab if len(elem) >= 5]
print len(vocab)

X = model[vocab]

In [ ]:
proj = TSNE(n_components=2, random_state=0)
np.set_printoptions(suppress=True)
Y = proj.fit_transform(X[:500]) 

plt.figure(figsize=(25,25))
plt.scatter(Y[:, 0], Y[:, 1],color='white')
for label, x, y in zip(vocab, Y[:, 0], Y[:, 1]):
    plt.annotate(label, xy=(x, y), xytext=(0, 0), textcoords='offset points')
plt.show()